In [28]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


In [29]:
from datetime import datetime

def NFTSoldProfitQuery(start_dt,end_dt):
    sql = """
insert into dw.`dm_nft_sold_profit_eth_hi_new` (
    `sell_transaction_hash`,
    `sell_block_number`,
    `sell_log_index`,
    `sell_ts`,
    `account_address`,
    `token_address`,
    `token_id`,
    `erc_standard`,
    `sell_token_num`,
    `sell_eth_amt`,
    `sell_usd_amt`,
    `sell_gas_cost`,
    `seller_receive_amount`,
    `buy_ts`,
    `buy_token_num`,
    `buy_eth_amt`,
    `buy_transaction_hash`,
    `buy_block_number`,
    `buy_type`,
    `profit`
)
with nft_sold_drv as (
    select from_address as account_address
        ,token_address
        ,token_ids
        ,token_num
        ,trade_eth_value
        ,trade_usd_value
        ,gas_cost
        ,trade_time
        ,transaction_hash
        ,log_index
        ,block_number
        ,'sell' as trade_type
        ,erc_standard
        ,seller_receive_amount
    from dw.dwb_nft_trade_eth_detail_hi
    where trade_time >= '{start_dt}'
    and trade_time <= '{end_dt}'
)
,nft_get_drv as (
    select to_address as account_address
        ,token_address
        ,token_ids
        ,token_num
        ,trade_eth_value as trade_eth_value
        ,trade_time as ts
        ,transaction_hash
        ,block_number
        ,'buy' as buy_type
    from dw.dwb_nft_trade_eth_detail_hi
    union all
    select account_address
        ,token_address
        ,token_id as token_ids
        ,mint_amount as token_num
        ,(gas_cost+mint_cost) as trade_eth_value
        ,mint_time as ts
        ,transaction_hash
        ,block_number
        ,'mint' as buy_type
    from dw.dwb_nft_mint_detail_eth_hi
)
select 
    `sell_transaction_hash`,
    `sell_block_number`,
    `sell_log_index`,
    `sell_ts`,
    `account_address`,
    `token_address`,
    `token_id`,
    `erc_standard`,
    `sell_token_num`,
    `sell_eth_amt`,
    `sell_usd_amt`,
    `sell_gas_cost`,
    `seller_receive_amount`,
    `buy_ts`,
    `buy_token_num`,
    `buy_eth_amt`,
    `buy_transaction_hash`,
    `buy_block_number`,
    `buy_type`,
    `profit`
from
(
    select a.account_address
        ,a.token_address
        ,a.token_ids as token_id
        ,a.erc_standard
        ,a.trade_time as sell_ts
        ,a.trade_eth_value as sell_eth_amt
        ,a.transaction_hash as sell_transaction_hash
        ,a.block_number as sell_block_number
        ,a.log_index as sell_log_index
        ,a.token_num as sell_token_num
        ,a.trade_usd_value as sell_usd_amt
        ,a.gas_cost as sell_gas_cost
        ,a.seller_receive_amount

        ,b.ts as buy_ts
        ,b.token_num as buy_token_num
        ,b.trade_eth_value as buy_eth_amt
        ,b.transaction_hash as buy_transaction_hash
        ,b.block_number as buy_block_number
        ,b.buy_type
        ,COALESCE(a.trade_eth_value,0)-coalesce(b.trade_eth_value,0) as profit
        ,ROW_NUMBER()over(partition by a.transaction_hash,a.log_index,a.block_number,a.trade_time order by b.ts desc) as sold_pre_buy_rnk
    from nft_sold_drv as a
    left join nft_get_drv as b
    on a.token_address = b.token_address
    and a.token_ids = b.token_ids
    and a.trade_time > b.ts
) as drv
where sold_pre_buy_rnk = 1
""".format(
        start_dt = start_dt,
        end_dt = end_dt
    )
    return sql

In [ ]:
from datetime import datetime
from datetime import timedelta

# end_dt = datetime.now().date()
end_dt = datetime(2022,1,1).date()

while end_dt > datetime(2017,1,1).date():
    if end_dt > datetime(2021,8,1).date():
        start_dt = end_dt - timedelta(days=15)
    elif end_dt > datetime(2020,6,1).date():
        start_dt = end_dt - timedelta(days=30)
    else:
        start_dt = end_dt - timedelta(days=180)
    print(start_dt,end_dt)
    start_str = start_dt.strftime('%Y-%m-%d')
    end_str = end_dt.strftime('%Y-%m-%d')
    sql = NFTSoldProfitQuery(start_str,end_str)
    cursor.execute(sql)
    end_dt = start_dt


start_dt = datetime(2016,1,1)
print(start_dt,end_dt)
start_str = start_dt.strftime('%Y-%m-%d')
end_str = end_dt.strftime('%Y-%m-%d')
sql = NFTSoldProfitQuery(start_str,end_str)
cursor.execute(sql)


2021-12-17 2022-01-01


In [ ]:

DROP TABLE IF EXISTS dw.`dm_nft_sold_profit_eth_hi_new`;
create table dw.`dm_nft_sold_profit_eth_hi_new` (
    `sell_transaction_hash` varchar(80) NOT NULL COMMENT "nft卖出transaction hash",
		`sell_block_number` bigint(20) NOT NULL COMMENT "nft卖出交易的block_number",
		`sell_log_index` int(11) NOT NULL COMMENT "nft卖出交易的log_index",
		`sell_ts` datetime NOT NULL COMMENT "nft卖出时间",
		`account_address` varchar(42) NOT NULL COMMENT "the account_address sold the nft",
		`token_address` varchar(65533) NOT NULL COMMENT "the nft token_address been sold",
		`token_id` varchar(65533) NULL COMMENT "the nft token_id been sold",
		`erc_standard` varchar(20) NULL COMMENT "",
		`sell_token_num` bigint(20) NULL COMMENT "卖出nft数量",
		`sell_eth_amt` double NULL COMMENT "该笔卖出交易eth价值",
		`sell_usd_amt` double NULL COMMENT "该笔卖出交易USD价值",
		`sell_gas_cost` double NULL COMMENT "该笔卖出交易花费Gas /eth",
		`seller_receive_amount` double NULL COMMENT "该笔卖出交易 卖家收入的金额",
		`buy_ts` varchar(255) NULL COMMENT "卖出之前的一笔买入的 买入时间",
		`buy_token_num` bigint(20) NULL COMMENT "卖出之前的一笔买入的 nft数量",
		`buy_eth_amt` double NULL COMMENT "卖出之前的一笔买入的 eth价值",
		`buy_transaction_hash` varchar(80) NULL COMMENT "卖出之前的一笔买入的 transaction hash",
		`buy_block_number` varchar(80) NULL COMMENT "nft卖出交易的block_number",
		`buy_type` varchar(10) NULL COMMENT "入手方式: buy/mint",
		`profit` double NULL COMMENT "盈利"
)ENGINE=OLAP
UNIQUE KEY(`sell_transaction_hash`,`sell_block_number`,`sell_log_index`,`sell_ts`)
COMMENT ""
PARTITION BY RANGE(`sell_ts`)
(
partition p201701 VALUES [('2016-01-01 00:00:00'), ('2017-01-01 00:00:00')),
partition p201701 VALUES [('2017-01-01 00:00:00'), ('2018-01-01 00:00:00')),
PARTITION p201801 VALUES [('2018-01-01 00:00:00'), ('2019-01-01 00:00:00')),
PARTITION p201901 VALUES [('2019-01-01 00:00:00'), ('2020-01-01 00:00:00')),
PARTITION p202001 VALUES [('2020-01-01 00:00:00'), ('2021-01-01 00:00:00')),
PARTITION p202101 VALUES [('2021-01-01 00:00:00'), ('2021-03-01 00:00:00')),
PARTITION p202103 VALUES [('2021-03-01 00:00:00'), ('2021-05-01 00:00:00')),
PARTITION p202105 VALUES [('2021-05-01 00:00:00'), ('2021-07-01 00:00:00')),
PARTITION p202107 VALUES [('2021-07-01 00:00:00'), ('2021-09-01 00:00:00')),
PARTITION p202109 VALUES [('2021-09-01 00:00:00'), ('2021-11-01 00:00:00')),
PARTITION p2021011 VALUES [('2021-11-01 00:00:00'), ('2022-01-01 00:00:00')),
PARTITION p202201 VALUES [('2022-01-01 00:00:00'), ('2022-02-01 00:00:00')),
PARTITION p202202 VALUES [('2022-02-01 00:00:00'), ('2022-03-01 00:00:00')),
PARTITION p202203 VALUES [('2022-03-01 00:00:00'), ('2022-04-01 00:00:00')),
PARTITION p202204 VALUES [('2022-04-01 00:00:00'), ('2022-05-01 00:00:00')),
PARTITION p202205 VALUES [('2022-05-01 00:00:00'), ('2022-06-01 00:00:00')),
PARTITION p202206 VALUES [('2022-06-01 00:00:00'), ('2022-07-01 00:00:00')),
PARTITION p202207 VALUES [('2022-07-01 00:00:00'), ('2022-08-01 00:00:00')),
PARTITION p202208 VALUES [('2022-08-01 00:00:00'), ('2022-09-01 00:00:00')),
PARTITION p202209 VALUES [('2022-09-01 00:00:00'), ('2022-10-01 00:00:00')),
PARTITION p202210 VALUES [('2022-10-01 00:00:00'), ('2022-11-01 00:00:00')),
PARTITION p202211 VALUES [('2022-11-01 00:00:00'), ('2022-12-01 00:00:00')),
PARTITION p202212 VALUES [('2022-12-01 00:00:00'), ('2023-01-01 00:00:00')))
DISTRIBUTED BY HASH(`sell_transaction_hash`) BUCKETS 15
PROPERTIES (
"replication_num" = "3",
"dynamic_partition.enable" = "true",
"dynamic_partition.time_unit" = "MONTH",
"dynamic_partition.time_zone" = "Etc/UTC",
"dynamic_partition.start" = "-2147483648",
"dynamic_partition.end" = "2",
"dynamic_partition.prefix" = "p",
"dynamic_partition.buckets" = "32",
"dynamic_partition.replication_num" = "3",
"dynamic_partition.start_day_of_month" = "1",
"in_memory" = "false",
"storage_format" = "V2"
);
